## Instruction
1. Comparision for batch of images, need to specify the path

In [4]:
%load_ext autoreload
%autoreload 2
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


/home/lincolnzjx/anaconda3/envs/Py36/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['std', 'mean']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


## 0. Import module

In [5]:
import sys
sys.path.append("../../../src/")
sys.path.append("../../")

from torchvision import transforms
# from aux.visualization import visualize_features_map_for_comparision
from aux.utils import obtain_features_map, load_imgs, zscore, extract_valid
import config
from loss import FilterLoss
from datasets import imagenet
import model
import sys
from PIL import Image
import matplotlib as mpl
from torchsummary import summary
import torch
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

mpl.rcParams['figure.dpi'] = 600

In [6]:
# Load data
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
reverse_mean = [-0.485, -0.456, -0.406]
reverse_std = [1/0.229, 1/0.224, 1/0.225]
train_transform = None
train_transform = transforms.Compose([                                                                                                                                
    transforms.Resize((224, 224), interpolation=Image.BILINEAR),                                                                                              
    transforms.ToTensor(),                                                                                                                                            
    transforms.Normalize(mean, std)                                                                                                                                   
       ])         
trainset = imagenet.ImageNet(root="/media/lincolnzjx/HardDisk/Datasets/", is_train=True, transform=train_transform)

trainset.set_data([950], 30)
imgs_path = []                                                                                                                                                            
images = []
labels = []
for img, label, img_path in trainset:                                                                                                                                     
    images.append(img.unsqueeze(0))                                                                                                                                       
    labels.append(label)                                                                                                                                                  
    imgs_path.append(img_path)  

Len of new dataset is :30


In [31]:
save_dir = "/home/lincolnzjx/Desktop/Interpretation/saved/pack"
################### Hyper-Parameter #######################
exp = "033004"
epoch = "99900"
##########################################################
ab_path = os.path.join(save_dir, exp, epoch)

In [32]:
resume = "037-0"
model_dir = "/home/lincolnzjx/Desktop/Interpretation/saved/models"
backbone = "vgg16"

# Load model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = model.Network(backbone=backbone, num_classes=1000, selected_layer=1)
net.to(device)
# resume from model
resume_exp = resume.split("-")[0]
resume_epoch = resume.split("-")[1]
print("Resume from model from exp: {} at epoch {}".format(resume_exp, resume_epoch))
resume_path = os.path.join(model_dir, str(resume_exp), str(resume_epoch))
ckpt = torch.load(resume_path, map_location=device)
net.load_state_dict(ckpt, strict=False) 

Only Keep 1th layers before.
Resume from model from exp: 037 at epoch 0


_IncompatibleKeys(missing_keys=[], unexpected_keys=['model.fc.0.weight', 'model.fc.0.bias', 'model.fc.3.weight', 'model.fc.3.bias', 'model.fc.6.weight', 'model.fc.6.bias', 'model.features.2.weight', 'model.features.2.bias', 'model.features.5.weight', 'model.features.5.bias', 'model.features.7.weight', 'model.features.7.bias', 'model.features.10.weight', 'model.features.10.bias', 'model.features.12.weight', 'model.features.12.bias', 'model.features.14.weight', 'model.features.14.bias', 'model.features.17.weight', 'model.features.17.bias', 'model.features.19.weight', 'model.features.19.bias', 'model.features.21.weight', 'model.features.21.bias', 'model.features.24.weight', 'model.features.24.bias', 'model.features.26.weight', 'model.features.26.bias', 'model.features.28.weight', 'model.features.28.bias'])

In [33]:
net

Network(
  (model): VGG16(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
    )
  )
)

In [34]:
# Load image
optimized_data, valid_imgs_path, valid_imgs_index = load_imgs(ab_path, imgs_path, non_exists_ok=True)
valid_imgs, valid_labels = extract_valid(images, labels, valid_imgs_index)
optimized_data = zscore(optimized_data, mean, std)

Load from /home/lincolnzjx/Desktop/Interpretation/saved/pack/033004/99900/n07747607_10061.JPEG
Load from /home/lincolnzjx/Desktop/Interpretation/saved/pack/033004/99900/n07747607_10068.JPEG
Load from /home/lincolnzjx/Desktop/Interpretation/saved/pack/033004/99900/n07747607_10110.JPEG
Load from /home/lincolnzjx/Desktop/Interpretation/saved/pack/033004/99900/n07747607_10134.JPEG
Load from /home/lincolnzjx/Desktop/Interpretation/saved/pack/033004/99900/n07747607_10162.JPEG
Load from /home/lincolnzjx/Desktop/Interpretation/saved/pack/033004/99900/n07747607_10166.JPEG
Load from /home/lincolnzjx/Desktop/Interpretation/saved/pack/033004/99900/n07747607_10178.JPEG
Load from /home/lincolnzjx/Desktop/Interpretation/saved/pack/033004/99900/n07747607_10179.JPEG
Load from /home/lincolnzjx/Desktop/Interpretation/saved/pack/033004/99900/n07747607_10184.JPEG
Load from /home/lincolnzjx/Desktop/Interpretation/saved/pack/033004/99900/n07747607_10192.JPEG
Load from /home/lincolnzjx/Desktop/Interpretation/

In [35]:
# Move to device
opt_image = torch.from_numpy(optimized_data).to(device)
original_image = torch.cat(valid_imgs, dim=0).to(device)

In [36]:
selected_layer = 1
selected_filter = 47
mode = "keep"
inter = False
rho = 0
regularization = "L1"
smoothing = "None"
regular_ex = 2
alpha = 1
beta = 1
gamma = 1
delta = 0

In [37]:
# 2.Calculate Loss
criterion = FilterLoss(net, selected_layer, selected_filter, mode,
                        inter=inter, rho=rho,
                        regularization=regularization,
                        smoothing=smoothing, p=regular_ex, _print=print)

In [38]:
selected_filter_loss, rest_fileter_loss, regularization_loss,  \
      smoothing_loss = criterion(opt_image, original_image)
print(selected_filter_loss)
print(rest_fileter_loss)
print(regularization_loss)
print(smoothing_loss)

tensor(0.0458, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.2403, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.2797, device='cuda:0')
0.0


In [22]:
import numpy as np

In [15]:
matrix = np.load(os.path.join(ab_path, "45000.npy"))

In [16]:
#del selected_filter_loss, rest_fileter_loss, regularization_loss, smoothing_loss
opt_image = torch.from_numpy(matrix).to(device)
selected_filter_loss, rest_fileter_loss, regularization_loss,  \
      smoothing_loss = criterion(opt_image, original_image)
print(selected_filter_loss)
print(rest_fileter_loss)
print(regularization_loss)
print(smoothing_loss)

tensor(0.0091, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.3533, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0204, device='cuda:0')
0.0


In [20]:
np.mean(np.abs(optimized_data))

0.021081386

In [21]:
np.mean(np.abs(matrix))

0.02035715

In [19]:
net

Network(
  (model): VGG16(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16)

In [46]:
optimized_data - matrix

ValueError: invalid __array_struct__

type

numpy.matrix